In [29]:
import pandas as pd
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')

## Uploading raw data into the MySQL database

In [6]:
import os
os.path.abspath("")

'C:\\Users\\Aniket\\Documents\\Analytics programming and data visualization\\APDV 65 group project'

In [12]:
dat = pd.read_csv("owid-energy-data.csv")

In [31]:
import mysql.connector     # Python's module for connecting and accessing a MySQL database  

In [104]:
conn = mysql.connector.connect(host="localhost", user="root", password="Anirane15@isro",database="")
cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS energy_database")

print("Database 'energy_database' created successfully!")

# Close connection
cursor.close()
conn.close()

Database 'energy_database' created successfully!


In [392]:
# Creating a table for maintaining the raw unprocessed data
conn = mysql.connector.connect(host="localhost", user="root", password="Anirane15@isro", database="energy_database")
cursor = conn.cursor()
# Creating the 'raw_energy_stats' table for unprocessed data
cursor.execute("""
    CREATE TABLE IF NOT EXISTS raw_energy_stats (
        iso_code VARCHAR(10),
        country VARCHAR(255),
        year INT NOT NULL,
        population BIGINT,
        gdp BIGINT, biofuel_cons_change_pct FLOAT, biofuel_cons_change_twh FLOAT, biofuel_cons_per_capita FLOAT, biofuel_consumption FLOAT, 
        biofuel_elec_per_capita FLOAT, biofuel_electricity FLOAT, biofuel_share_elec FLOAT, biofuel_share_energy FLOAT, carbon_intensity_elec FLOAT, 
        coal_cons_change_pct FLOAT, coal_cons_change_twh FLOAT, coal_cons_per_capita FLOAT, coal_consumption FLOAT, coal_elec_per_capita FLOAT, coal_electricity FLOAT, 
        coal_prod_change_pct FLOAT, coal_prod_change_twh FLOAT, coal_prod_per_capita FLOAT, coal_production FLOAT, coal_share_elec FLOAT, coal_share_energy FLOAT, 
        electricity_demand FLOAT, electricity_generation FLOAT, electricity_share_energy FLOAT, energy_cons_change_pct FLOAT, 
        energy_cons_change_twh FLOAT, energy_per_capita FLOAT, energy_per_gdp FLOAT, fossil_cons_change_pct FLOAT, fossil_cons_change_twh FLOAT, 
        fossil_elec_per_capita FLOAT, fossil_electricity FLOAT, fossil_energy_per_capita FLOAT, fossil_fuel_consumption FLOAT, fossil_share_elec FLOAT, 
        fossil_share_energy FLOAT, gas_cons_change_pct FLOAT, gas_cons_change_twh FLOAT, gas_consumption FLOAT, gas_elec_per_capita FLOAT, gas_electricity FLOAT, 
        gas_energy_per_capita FLOAT, gas_prod_change_pct FLOAT, gas_prod_change_twh FLOAT, gas_prod_per_capita FLOAT, gas_production FLOAT, gas_share_elec FLOAT, gas_share_energy FLOAT, 
        greenhouse_gas_emissions FLOAT, hydro_cons_change_pct FLOAT, hydro_cons_change_twh FLOAT, hydro_consumption FLOAT, hydro_elec_per_capita FLOAT, hydro_electricity FLOAT, hydro_energy_per_capita FLOAT, hydro_share_elec FLOAT, hydro_share_energy FLOAT, 
        low_carbon_cons_change_pct FLOAT, low_carbon_cons_change_twh FLOAT, low_carbon_consumption FLOAT, low_carbon_elec_per_capita FLOAT, low_carbon_electricity FLOAT, low_carbon_energy_per_capita FLOAT, low_carbon_share_elec FLOAT, low_carbon_share_energy FLOAT, 
        net_elec_imports FLOAT, net_elec_imports_share_demand FLOAT, nuclear_cons_change_pct FLOAT, nuclear_cons_change_twh FLOAT, nuclear_consumption FLOAT, nuclear_elec_per_capita FLOAT, nuclear_electricity FLOAT, 
        nuclear_energy_per_capita FLOAT, nuclear_share_elec FLOAT, nuclear_share_energy FLOAT, oil_cons_change_pct FLOAT, 
        oil_cons_change_twh FLOAT, oil_consumption FLOAT, oil_elec_per_capita FLOAT, oil_electricity FLOAT, oil_energy_per_capita FLOAT, 
        oil_prod_change_pct FLOAT, oil_prod_change_twh FLOAT, oil_prod_per_capita FLOAT, oil_production FLOAT, oil_share_elec FLOAT, oil_share_energy FLOAT, 
        other_renewable_consumption FLOAT, other_renewable_electricity FLOAT, other_renewable_exc_biofuel_electricity FLOAT, other_renewables_cons_change_pct FLOAT, other_renewables_cons_change_twh FLOAT, other_renewables_elec_per_capita FLOAT, other_renewables_elec_per_capita_exc_biofuel FLOAT, other_renewables_energy_per_capita FLOAT, other_renewables_share_elec FLOAT, other_renewables_share_elec_exc_biofuel FLOAT, other_renewables_share_energy FLOAT, 
        per_capita_electricity FLOAT, primary_energy_consumption FLOAT, renewables_cons_change_pct FLOAT, renewables_cons_change_twh FLOAT, renewables_consumption FLOAT, renewables_elec_per_capita FLOAT, renewables_electricity FLOAT, 
        renewables_energy_per_capita FLOAT, renewables_share_elec FLOAT, renewables_share_energy FLOAT, solar_cons_change_pct FLOAT, solar_cons_change_twh FLOAT, solar_consumption FLOAT, solar_elec_per_capita FLOAT, solar_electricity FLOAT, solar_energy_per_capita FLOAT, solar_share_elec FLOAT, solar_share_energy FLOAT, wind_cons_change_pct FLOAT, wind_cons_change_twh FLOAT, wind_consumption FLOAT, wind_elec_per_capita FLOAT, wind_electricity FLOAT, wind_energy_per_capita FLOAT, wind_share_elec FLOAT, wind_share_energy FLOAT
    )
""")

In [394]:
# Creating a bulk function to insert the raw data
from tqdm import tqdm
def insert_energy_stats_bulk(df, cursor, conn, columns=None, batch_size=500):

    # Default energy columns if not specified
    if columns is None:
        columns = ['country', 'year', 'iso_code', 'population', 'gdp', 'biofuel_cons_change_pct', 'biofuel_cons_change_twh', 'biofuel_cons_per_capita', 'biofuel_consumption', 'biofuel_elec_per_capita', 'biofuel_electricity', 'biofuel_share_elec', 'biofuel_share_energy', 'carbon_intensity_elec', 'coal_cons_change_pct', 'coal_cons_change_twh', 'coal_cons_per_capita', 'coal_consumption', 'coal_elec_per_capita', 'coal_electricity', 'coal_prod_change_pct', 'coal_prod_change_twh', 'coal_prod_per_capita', 'coal_production', 'coal_share_elec', 'coal_share_energy', 'electricity_demand', 'electricity_generation', 'electricity_share_energy', 'energy_cons_change_pct', 'energy_cons_change_twh', 'energy_per_capita', 'energy_per_gdp', 'fossil_cons_change_pct', 'fossil_cons_change_twh', 'fossil_elec_per_capita', 'fossil_electricity', 'fossil_energy_per_capita', 'fossil_fuel_consumption', 'fossil_share_elec', 'fossil_share_energy', 'gas_cons_change_pct', 'gas_cons_change_twh', 'gas_consumption', 'gas_elec_per_capita', 'gas_electricity', 'gas_energy_per_capita', 'gas_prod_change_pct', 'gas_prod_change_twh', 'gas_prod_per_capita', 'gas_production', 'gas_share_elec', 'gas_share_energy', 'greenhouse_gas_emissions', 'hydro_cons_change_pct', 'hydro_cons_change_twh', 'hydro_consumption', 'hydro_elec_per_capita', 'hydro_electricity', 'hydro_energy_per_capita', 'hydro_share_elec', 'hydro_share_energy', 'low_carbon_cons_change_pct', 'low_carbon_cons_change_twh', 'low_carbon_consumption', 'low_carbon_elec_per_capita', 'low_carbon_electricity', 'low_carbon_energy_per_capita', 'low_carbon_share_elec', 'low_carbon_share_energy', 'net_elec_imports', 'net_elec_imports_share_demand', 'nuclear_cons_change_pct', 'nuclear_cons_change_twh', 'nuclear_consumption', 'nuclear_elec_per_capita', 'nuclear_electricity', 'nuclear_energy_per_capita', 'nuclear_share_elec', 'nuclear_share_energy', 'oil_cons_change_pct', 'oil_cons_change_twh', 'oil_consumption', 'oil_elec_per_capita', 'oil_electricity', 'oil_energy_per_capita', 'oil_prod_change_pct', 'oil_prod_change_twh', 'oil_prod_per_capita', 'oil_production', 'oil_share_elec', 'oil_share_energy', 'other_renewable_consumption', 'other_renewable_electricity', 'other_renewable_exc_biofuel_electricity', 'other_renewables_cons_change_pct', 'other_renewables_cons_change_twh', 'other_renewables_elec_per_capita', 'other_renewables_elec_per_capita_exc_biofuel', 'other_renewables_energy_per_capita', 'other_renewables_share_elec', 'other_renewables_share_elec_exc_biofuel', 'other_renewables_share_energy', 'per_capita_electricity', 'primary_energy_consumption', 'renewables_cons_change_pct', 'renewables_cons_change_twh', 'renewables_consumption', 'renewables_elec_per_capita', 'renewables_electricity', 'renewables_energy_per_capita', 'renewables_share_elec', 'renewables_share_energy', 'solar_cons_change_pct', 'solar_cons_change_twh', 'solar_consumption', 'solar_elec_per_capita', 'solar_electricity', 'solar_energy_per_capita', 'solar_share_elec', 'solar_share_energy', 'wind_cons_change_pct', 'wind_cons_change_twh', 'wind_consumption', 'wind_elec_per_capita', 'wind_electricity', 'wind_energy_per_capita', 'wind_share_elec', 'wind_share_energy']
    
    # Keep only relevant columns
    energy_df = df[columns].copy()
    energy_df = energy_df.replace({np.nan: None})
    
    print("Inserting energy stats into database...")

    placeholders = ', '.join(['%s'] * len(columns))
    col_names = ', '.join(columns)
    
    count = 0

    for _, row in tqdm(energy_df.iterrows(), total=len(energy_df)):
        try:
            values = [row[col] for col in columns]
            cursor.execute(f"""
                INSERT INTO raw_energy_stats ({col_names})
                VALUES ({placeholders})
            """, values)
            count += 1

            if count % batch_size == 0:
                conn.commit()

        except Exception as e:
            print(f"Error inserting row: {e}")

    conn.commit()
    print(f"Inserted successfully. Total rows inserted: {count}")

In [396]:
sel_cols = ['iso_code', 'country', 'year', 'population', 'gdp', 'biofuel_cons_change_pct', 'biofuel_cons_change_twh', 'biofuel_cons_per_capita', 'biofuel_consumption', 'biofuel_elec_per_capita', 'biofuel_electricity', 'biofuel_share_elec', 'biofuel_share_energy', 'carbon_intensity_elec', 'coal_cons_change_pct', 'coal_cons_change_twh', 'coal_cons_per_capita', 'coal_consumption', 'coal_elec_per_capita', 'coal_electricity', 'coal_prod_change_pct', 'coal_prod_change_twh', 'coal_prod_per_capita', 'coal_production', 'coal_share_elec', 'coal_share_energy', 'electricity_demand', 'electricity_generation', 'electricity_share_energy', 'energy_cons_change_pct', 'energy_cons_change_twh', 'energy_per_capita', 'energy_per_gdp', 'fossil_cons_change_pct', 'fossil_cons_change_twh', 'fossil_elec_per_capita', 'fossil_electricity', 'fossil_energy_per_capita', 'fossil_fuel_consumption', 'fossil_share_elec', 'fossil_share_energy', 'gas_cons_change_pct', 'gas_cons_change_twh', 'gas_consumption', 'gas_elec_per_capita', 'gas_electricity', 'gas_energy_per_capita', 'gas_prod_change_pct', 'gas_prod_change_twh', 'gas_prod_per_capita', 'gas_production', 'gas_share_elec', 'gas_share_energy', 'greenhouse_gas_emissions', 'hydro_cons_change_pct', 'hydro_cons_change_twh', 'hydro_consumption', 'hydro_elec_per_capita', 'hydro_electricity', 'hydro_energy_per_capita', 'hydro_share_elec', 'hydro_share_energy', 'low_carbon_cons_change_pct', 'low_carbon_cons_change_twh', 'low_carbon_consumption', 'low_carbon_elec_per_capita', 'low_carbon_electricity', 'low_carbon_energy_per_capita', 'low_carbon_share_elec', 'low_carbon_share_energy', 'net_elec_imports', 'net_elec_imports_share_demand', 'nuclear_cons_change_pct', 'nuclear_cons_change_twh', 'nuclear_consumption', 'nuclear_elec_per_capita', 'nuclear_electricity', 'nuclear_energy_per_capita', 'nuclear_share_elec', 'nuclear_share_energy', 'oil_cons_change_pct', 'oil_cons_change_twh', 'oil_consumption', 'oil_elec_per_capita', 'oil_electricity', 'oil_energy_per_capita', 'oil_prod_change_pct', 'oil_prod_change_twh', 'oil_prod_per_capita', 'oil_production', 'oil_share_elec', 'oil_share_energy', 'other_renewable_consumption', 'other_renewable_electricity', 'other_renewable_exc_biofuel_electricity', 'other_renewables_cons_change_pct', 'other_renewables_cons_change_twh', 'other_renewables_elec_per_capita', 'other_renewables_elec_per_capita_exc_biofuel', 'other_renewables_energy_per_capita', 'other_renewables_share_elec', 'other_renewables_share_elec_exc_biofuel', 'other_renewables_share_energy', 'per_capita_electricity', 'primary_energy_consumption', 'renewables_cons_change_pct', 'renewables_cons_change_twh', 'renewables_consumption', 'renewables_elec_per_capita', 'renewables_electricity', 'renewables_energy_per_capita', 'renewables_share_elec', 'renewables_share_energy', 'solar_cons_change_pct', 'solar_cons_change_twh', 'solar_consumption', 'solar_elec_per_capita', 'solar_electricity', 'solar_energy_per_capita', 'solar_share_elec', 'solar_share_energy', 'wind_cons_change_pct', 'wind_cons_change_twh', 'wind_consumption', 'wind_elec_per_capita', 'wind_electricity', 'wind_energy_per_capita', 'wind_share_elec', 'wind_share_energy']
insert_energy_stats_bulk(dat, cursor, conn, columns=sel_cols)

Inserting energy stats into database...


100%|██████████████████████████████████████████████████████████████████████████| 21812/21812 [00:13<00:00, 1604.51it/s]

Inserted successfully. Total rows inserted: 21812


## Calling the data from database for pre-processing

In [398]:
conn = mysql.connector.connect(host="localhost", user="root", password="Anirane15@isro",database="energy_database")

query = "SELECT * FROM raw_energy_stats"

In [400]:
data = pd.read_sql(query, con = conn)

In [401]:
data.head()

,iso_code,country,year,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,biofuel_share_energy,carbon_intensity_elec,coal_cons_change_pct,coal_cons_change_twh,coal_cons_per_capita,coal_consumption,coal_elec_per_capita,coal_electricity,coal_prod_change_pct,coal_prod_change_twh,coal_prod_per_capita,coal_production,coal_share_elec,coal_share_energy,electricity_demand,electricity_generation,electricity_share_energy,energy_cons_change_pct,energy_cons_change_twh,energy_per_capita,energy_per_gdp,fossil_cons_change_pct,fossil_cons_change_twh,fossil_elec_per_capita,fossil_electricity,fossil_energy_per_capita,fossil_fuel_consumption,fossil_share_elec,fossil_share_energy,gas_cons_change_pct,gas_cons_change_twh,gas_consumption,gas_elec_per_capita,gas_electricity,gas_energy_per_capita,gas_prod_change_pct,gas_prod_change_twh,gas_prod_per_capita,gas_production,gas_share_elec,gas_share_energy,greenhouse_gas_emissions,hydro_cons_change_pct,hydro_cons_change_twh,hydro_consumption,hydro_elec_per_capita,hydro_electricity,hydro_energy_per_capita,hydro_share_elec,hydro_share_energy,low_carbon_cons_change_pct,low_carbon_cons_change_twh,low_carbon_consumption,low_carbon_elec_per_capita,low_carbon_electricity,low_carbon_energy_per_capita,low_carbon_share_elec,low_carbon_share_energy,net_elec_imports,net_elec_imports_share_demand,nuclear_cons_change_pct,nuclear_cons_change_twh,nuclear_consumption,nuclear_elec_per_capita,nuclear_electricity,nuclear_energy_per_capita,nuclear_share_elec,nuclear_share_energy,oil_cons_change_pct,oil_cons_change_twh,oil_consumption,oil_elec_per_capita,oil_electricity,oil_energy_per_capita,oil_prod_change_pct,oil_prod_change_twh,oil_prod_per_capita,oil_production,oil_share_elec,oil_share_energy,other_renewable_consumption,other_renewable_electricity,other_renewable_exc_biofuel_electricity,other_renewables_cons_change_pct,other_renewables_cons_change_twh,other_renewables_elec_per_capita,other_renewables_elec_per_capita_exc_biofuel,other_renewables_energy_per_capita,other_renewables_share_elec,other_renewables_share_elec_exc_biofuel,other_renewables_share_energy,per_capita_electricity,primary_energy_consumption,renewables_cons_change_pct,renewables_cons_change_twh,renewables_consumption,renewables_elec_per_capita,renewables_electricity,renewables_energy_per_capita,renewables_share_elec,renewables_share_energy,solar_cons_change_pct,solar_cons_change_twh,solar_consumption,solar_elec_per_capita,solar_electricity,solar_energy_per_capita,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,None,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.87,1.550,NaN,569.557,NaN,NaN,NaN,NaN,NaN,76.03,NaN,NaN,NaN,NaN,20.081,NaN,378.61,378.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,305.36,NaN,NaN,80.653,NaN,NaN,NaN,NaN,NaN,164.26,NaN,NaN,NaN,NaN,NaN,43.385,NaN,215.64,NaN,NaN,NaN,NaN,50.45,NaN,13.325,NaN,NaN,NaN,NaN,NaN,73.25,NaN,19.347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,65.07,NaN,NaN,NaN,NaN,NaN,17.187,NaN,NaN,22.80,16.93,NaN,NaN,NaN,NaN,NaN,6.022,4.472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.25,NaN,19.347,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
1,None,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.46,1.596,NaN,567.642,NaN,NaN,NaN,NaN,NaN,86.26,NaN,NaN,NaN,NaN,21.307,NaN,404.85,404.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,327.66,NaN,NaN,80.934,NaN,NaN,NaN,NaN,NaN,190.41,NaN,NaN,NaN,NaN,NaN,47.032,NaN,229.81,NaN,NaN,NaN,NaN,54.33,NaN,13.420,NaN,NaN,NaN,NaN,NaN,77.19,NaN,19.066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,50.99,NaN,NaN,NaN,NaN,NaN,12.595,NaN,NaN,22.86,16.40,NaN,NaN,NaN,NaN,NaN,5.647,4.051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.19,NaN,19.066,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
2,None,ASEAN (Ember),2002,NaN,NaN

In [404]:
data.shape

(21812, 129)

In [325]:
#data = pd.read_csv("D:\\NCI Docs\\NCI LECS\\Analytics programming and data visualization\\Group project\\owid-energy-data.csv")
#data.head()

## Data Pre-processing

In [406]:
data_year = data.groupby("year")

In [408]:
data_year['year'].value_counts()

year
1900    118
1901    118
1902    118
1903    118
1904    118
       ... 
2019    274
2020    273
2021    273
2022    249
2023    155
Name: count, Length: 124, dtype: int64

#### We will filter the dataset for years and take data recorded in the 21st century into account as due to lack of technology and record keeping facilities the data for earlier years is very scarce.

In [411]:
data_1 = data[data['year'] >= 2000]

In [413]:
data_1.shape

(6659, 129)

In [415]:
data_1.isna().sum()

iso_code                  1508
country                      0
year                         0
population                1223
gdp                       2854
                          ... 
wind_elec_per_capita      1396
wind_electricity           820
wind_energy_per_capita    4501
wind_share_elec            848
wind_share_energy         4464
Length: 129, dtype: int64

#### From the above output,it can be noticed that many features have a lot of missing values. We will filter the data and include only those featues for which the percentage of missing values is less than 60%

In [418]:
((data_1.isna().sum()/len(data_1)) * 100).sort_values(ascending=False)

nuclear_cons_change_pct       82.459829
biofuel_cons_change_pct       77.414026
biofuel_cons_per_capita       76.618111
biofuel_share_energy          73.359363
biofuel_cons_change_twh       73.104070
                                ...    
electricity_generation        11.983781
energy_cons_change_twh        11.908695
primary_energy_consumption    11.833609
year                           0.000000
country                        0.000000
Length: 129, dtype: float64

In [420]:
miss_per = (data_1.isna().sum() / len(data_1)) * 100
sel_col = miss_per[miss_per < 60]

In [422]:
data_2 = data_1[sel_col.index]

In [424]:
data_2.head()

,iso_code,country,year,population,gdp,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,carbon_intensity_elec,coal_elec_per_capita,coal_electricity,coal_prod_change_twh,coal_prod_per_capita,coal_production,coal_share_elec,electricity_demand,electricity_generation,energy_cons_change_pct,energy_cons_change_twh,energy_per_capita,energy_per_gdp,fossil_elec_per_capita,fossil_electricity,fossil_share_elec,gas_elec_per_capita,gas_electricity,gas_prod_change_twh,gas_prod_per_capita,gas_production,gas_share_elec,greenhouse_gas_emissions,hydro_elec_per_capita,hydro_electricity,hydro_share_elec,low_carbon_elec_per_capita,low_carbon_electricity,low_carbon_share_elec,net_elec_imports,net_elec_imports_share_demand,nuclear_cons_change_twh,nuclear_consumption,nuclear_elec_per_capita,nuclear_electricity,nuclear_energy_per_capita,nuclear_share_elec,oil_elec_per_capita,oil_electricity,oil_prod_change_pct,oil_prod_change_twh,oil_prod_per_capita,oil_production,oil_share_elec,other_renewable_electricity,other_renewable_exc_biofuel_electricity,other_renewables_elec_per_capita,other_renewables_elec_per_capita_exc_biofuel,other_renewables_share_elec,other_renewables_share_elec_exc_biofuel,per_capita_electricity,primary_energy_consumption,renewables_elec_per_capita,renewables_electricity,renewables_share_elec,solar_elec_per_capita,solar_electricity,solar_share_elec,wind_elec_per_capita,wind_electricity,wind_share_elec
0,None,ASEAN (Ember),2000,NaN,NaN,NaN,5.87,1.550,569.557,NaN,76.03,NaN,NaN,NaN,20.081,378.61,378.61,NaN,NaN,NaN,NaN,NaN,305.36,80.653,NaN,164.26,NaN,NaN,NaN,43.385,215.64,NaN,50.45,13.325,NaN,73.25,19.347,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,65.07,NaN,NaN,NaN,NaN,17.187,22.80,16.93,NaN,NaN,6.022,4.472,NaN,NaN,NaN,73.25,19.347,NaN,0.0,0.0,NaN,0.0,0.0
1,None,ASEAN (Ember),2001,NaN,NaN,NaN,6.46,1.596,567.642,NaN,86.26,NaN,NaN,NaN,21.307,404.85,404.85,NaN,NaN,NaN,NaN,NaN,327.66,80.934,NaN,190.41,NaN,NaN,NaN,47.032,229.81,NaN,54.33,13.420,NaN,77.19,19.066,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,50.99,NaN,NaN,NaN,NaN,12.595,22.86,16.40,NaN,NaN,5.647,4.051,NaN,NaN,NaN,77.19,19.066,NaN,0.0,0.0,NaN,0.0,0.0
2,None,ASEAN (Ember),2002,NaN,NaN,NaN,6.62,1.528,570.212,NaN,93.43,NaN,NaN,NaN,21.568,433.19,433.19,NaN,NaN,NaN,NaN,NaN,356.67,82.336,NaN,208.92,NaN,NaN,NaN,48.228,247.01,NaN,53.29,12.302,NaN,76.52,17.664,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,54.32,NaN,NaN,NaN,NaN,12.540,23.23,16.61,NaN,NaN,5.363,3.834,NaN,NaN,NaN,76.52,17.664,NaN,0.0,0.0,NaN,0.0,0.0
3,None,ASEAN (Ember),2003,NaN,NaN,NaN,7.45,1.626,574.699,NaN,102.01,NaN,NaN,NaN,22.261,458.24,458.24,NaN,NaN,NaN,NaN,NaN,381.84,83.328,NaN,226.51,NaN,NaN,NaN,49.430,263.35,NaN,53.21,11.612,NaN,76.40,16.672,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,53.32,NaN,NaN,NaN,NaN,11.636,23.19,15.74,NaN,NaN,5.061,3.435,NaN,NaN,NaN,76.40,16.672,NaN,0.0,0.0,NaN,0.0,0.0
4,None,ASEAN (Ember),2004,NaN,NaN,NaN,8.40,1.692,581.420,NaN,115.01,NaN,NaN,NaN,23.161,496.56,496.56,NaN,NaN,NaN,NaN,NaN,418.60,84.300,NaN,244.16,NaN,NaN,NaN,49.170,288.71,NaN,52.99,10.671,NaN,77.96,15.700,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,59.43,NaN,NaN,NaN,NaN,11.968,24.97,16.57,NaN,NaN,5.029,3.337,NaN,NaN,NaN,77.96,15.700,NaN,0.0,0.0,NaN,0.0,0.0


In [426]:
data_2.shape

(6659, 69)

In [428]:
# Drop rows(countries) where population count is not available
data_3 = data_2.dropna(subset=['population'])

In [430]:
data_3.shape

(5436, 69)

In [432]:
# Drop rows(countries) where iso_code is not available
data_4 = data_3.dropna(subset=['iso_code'])

In [434]:
data_4.shape

(5118, 69)

In [436]:
data_4['country'] = data_4['country'].str.strip().str.title()

In [438]:
data_4.isnull().sum()

iso_code                   0
country                    0
year                       0
population                 0
gdp                     1323
                        ... 
solar_electricity        139
solar_share_elec         162
wind_elec_per_capita     156
wind_electricity         156
wind_share_elec          184
Length: 69, dtype: int64

#### Selecting only the features that are important for analysis out of the remaining

In [441]:
df = data_4[['iso_code','country','year','population','biofuel_elec_per_capita','biofuel_electricity','biofuel_share_elec','carbon_intensity_elec','coal_elec_per_capita','coal_electricity','coal_share_elec','electricity_demand','electricity_generation','per_capita_electricity','fossil_elec_per_capita','fossil_electricity','fossil_share_elec','gas_elec_per_capita','gas_electricity','gas_share_elec','greenhouse_gas_emissions','low_carbon_electricity','low_carbon_elec_per_capita','low_carbon_share_elec','nuclear_elec_per_capita','nuclear_electricity','nuclear_share_elec','oil_elec_per_capita','oil_electricity','oil_share_elec','renewables_elec_per_capita','renewables_electricity','renewables_share_elec']]

In [443]:
df.head()

,iso_code,country,year,population,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,carbon_intensity_elec,coal_elec_per_capita,coal_electricity,coal_share_elec,electricity_demand,electricity_generation,per_capita_electricity,fossil_elec_per_capita,fossil_electricity,fossil_share_elec,gas_elec_per_capita,gas_electricity,gas_share_elec,greenhouse_gas_emissions,low_carbon_electricity,low_carbon_elec_per_capita,low_carbon_share_elec,nuclear_elec_per_capita,nuclear_electricity,nuclear_share_elec,oil_elec_per_capita,oil_electricity,oil_share_elec,renewables_elec_per_capita,renewables_electricity,renewables_share_elec
124,AFG,Afghanistan,2000,19542986.0,0.0,0.0,0.0,250.000,0.000,0.00,0.000,0.57,0.48,24.561,8.699,0.17,35.417,0.0,0.0,0.0,0.12,0.31,15.862,64.583,0.0,0.0,0.0,8.699,0.17,35.417,15.862,0.31,64.583
125,AFG,Afghanistan,2001,19688634.0,0.0,0.0,0.0,217.391,2.032,0.04,5.797,0.78,0.69,35.046,9.650,0.19,27.536,0.0,0.0,0.0,0.15,0.50,25.395,72.464,0.0,0.0,0.0,7.619,0.15,21.739,25.395,0.50,72.464
126,AFG,Afghanistan,2002,21000258.0,0.0,0.0,0.0,169.014,1.905,0.04,5.634,0.81,0.71,33.809,7.143,0.15,21.127,0.0,0.0,0.0,0.12,0.56,26.666,78.873,0.0,0.0,0.0,5.238,0.11,15.493,26.666,0.56,78.873
127,AFG,Afghanistan,2003,22645136.0,0.0,0.0,0.0,241.758,3.974,0.09,9.890,1.01,0.91,40.185,12.365,0.28,30.769,0.0,0.0,0.0,0.22,0.63,27.821,69.231,0.0,0.0,0.0,8.390,0.19,20.879,27.821,0.63,69.231
128,AFG,Afghanistan,2004,23553554.0,0.0,0.0,0.0,227.848,2.547,0.06,7.595,0.89,0.79,33.541,9.765,0.23,29.114,0.0,0.0,0.0,0.18,0.56,23.776,70.886,0.0,0.0,0.0,7.218,0.17,21.519,23.776,0.56,70.886


In [445]:
df.isnull().sum()

iso_code                        0
country                         0
year                            0
population                      0
biofuel_elec_per_capita       179
biofuel_electricity           179
biofuel_share_elec            202
carbon_intensity_elec         178
coal_elec_per_capita          219
coal_electricity              219
coal_share_elec               242
electricity_demand            150
electricity_generation        134
per_capita_electricity        134
fossil_elec_per_capita        146
fossil_electricity            146
fossil_share_elec             174
gas_elec_per_capita           261
gas_electricity               261
gas_share_elec                284
greenhouse_gas_emissions      150
low_carbon_electricity        135
low_carbon_elec_per_capita    135
low_carbon_share_elec         163
nuclear_elec_per_capita       186
nuclear_electricity           186
nuclear_share_elec            265
oil_elec_per_capita           151
oil_electricity               151
oil_share_elec

In [447]:
df.shape

(5118, 33)

#### Since, the number of missing values in the remaining features is very less as compared to the total number of rows in each column, we can replace the null values by calculating the mean for every year

In [450]:
iso_mapping = df[['country', 'iso_code']].drop_duplicates()

In [452]:
df_grouped = df.groupby(['country','year'], as_index=False).mean(numeric_only=True)

In [454]:
df_grouped.head()

,country,year,population,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,carbon_intensity_elec,coal_elec_per_capita,coal_electricity,coal_share_elec,electricity_demand,electricity_generation,per_capita_electricity,fossil_elec_per_capita,fossil_electricity,fossil_share_elec,gas_elec_per_capita,gas_electricity,gas_share_elec,greenhouse_gas_emissions,low_carbon_electricity,low_carbon_elec_per_capita,low_carbon_share_elec,nuclear_elec_per_capita,nuclear_electricity,nuclear_share_elec,oil_elec_per_capita,oil_electricity,oil_share_elec,renewables_elec_per_capita,renewables_electricity,renewables_share_elec
0,Afghanistan,2000,19542986.0,0.0,0.0,0.0,250.000,0.000,0.00,0.000,0.57,0.48,24.561,8.699,0.17,35.417,0.0,0.0,0.0,0.12,0.31,15.862,64.583,0.0,0.0,0.0,8.699,0.17,35.417,15.862,0.31,64.583
1,Afghanistan,2001,19688634.0,0.0,0.0,0.0,217.391,2.032,0.04,5.797,0.78,0.69,35.046,9.650,0.19,27.536,0.0,0.0,0.0,0.15,0.50,25.395,72.464,0.0,0.0,0.0,7.619,0.15,21.739,25.395,0.50,72.464
2,Afghanistan,2002,21000258.0,0.0,0.0,0.0,169.014,1.905,0.04,5.634,0.81,0.71,33.809,7.143,0.15,21.127,0.0,0.0,0.0,0.12,0.56,26.666,78.873,0.0,0.0,0.0,5.238,0.11,15.493,26.666,0.56,78.873
3,Afghanistan,2003,22645136.0,0.0,0.0,0.0,241.758,3.974,0.09,9.890,1.01,0.91,40.185,12.365,0.28,30.769,0.0,0.0,0.0,0.22,0.63,27.821,69.231,0.0,0.0,0.0,8.390,0.19,20.879,27.821,0.63,69.231
4,Afghanistan,2004,23553554.0,0.0,0.0,0.0,227.848,2.547,0.06,7.595,0.89,0.79,33.541,9.765,0.23,29.114,0.0,0.0,0.0,0.18,0.56,23.776,70.886,0.0,0.0,0.0,7.218,0.17,21.519,23.776,0.56,70.886


In [456]:
df_final = df_grouped.fillna(df_grouped.mean(numeric_only=True))

In [458]:
df_final = df_grouped.merge(iso_mapping, on='country', how='left')

#### Bringing the iso_code column to the front after the joining

In [461]:
last_col = df_final.columns[-1]

df_final = df_final[[last_col] + list(df_final.columns[:-1])]

In [463]:
df_final.head()

,iso_code,country,year,population,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,carbon_intensity_elec,coal_elec_per_capita,coal_electricity,coal_share_elec,electricity_demand,electricity_generation,per_capita_electricity,fossil_elec_per_capita,fossil_electricity,fossil_share_elec,gas_elec_per_capita,gas_electricity,gas_share_elec,greenhouse_gas_emissions,low_carbon_electricity,low_carbon_elec_per_capita,low_carbon_share_elec,nuclear_elec_per_capita,nuclear_electricity,nuclear_share_elec,oil_elec_per_capita,oil_electricity,oil_share_elec,renewables_elec_per_capita,renewables_electricity,renewables_share_elec
0,AFG,Afghanistan,2000,19542986.0,0.0,0.0,0.0,250.000,0.000,0.00,0.000,0.57,0.48,24.561,8.699,0.17,35.417,0.0,0.0,0.0,0.12,0.31,15.862,64.583,0.0,0.0,0.0,8.699,0.17,35.417,15.862,0.31,64.583
1,AFG,Afghanistan,2001,19688634.0,0.0,0.0,0.0,217.391,2.032,0.04,5.797,0.78,0.69,35.046,9.650,0.19,27.536,0.0,0.0,0.0,0.15,0.50,25.395,72.464,0.0,0.0,0.0,7.619,0.15,21.739,25.395,0.50,72.464
2,AFG,Afghanistan,2002,21000258.0,0.0,0.0,0.0,169.014,1.905,0.04,5.634,0.81,0.71,33.809,7.143,0.15,21.127,0.0,0.0,0.0,0.12,0.56,26.666,78.873,0.0,0.0,0.0,5.238,0.11,15.493,26.666,0.56,78.873
3,AFG,Afghanistan,2003,22645136.0,0.0,0.0,0.0,241.758,3.974,0.09,9.890,1.01,0.91,40.185,12.365,0.28,30.769,0.0,0.0,0.0,0.22,0.63,27.821,69.231,0.0,0.0,0.0,8.390,0.19,20.879,27.821,0.63,69.231
4,AFG,Afghanistan,2004,23553554.0,0.0,0.0,0.0,227.848,2.547,0.06,7.595,0.89,0.79,33.541,9.765,0.23,29.114,0.0,0.0,0.0,0.18,0.56,23.776,70.886,0.0,0.0,0.0,7.218,0.17,21.519,23.776,0.56,70.886


#### In the original data, we had data for continents as well which was dropped (except for Australia as it is both a country as well as a continent) as they had no iso_code mapping. 
#### So, now we will first create a separate dataframe and manually assign iso_codes for the continents Africa, Antarctica, Asia, Europe, North America, South America.

In [466]:
data_cont = data[(data['year'] >= 2000) & (data['country'].isin(['Africa','Antarctica','Asia','Europe','North America','South America']))]

#### Keeping only the selected columns as in the case of the data for countries

In [469]:
data_cont = data_cont[['iso_code','country','year','population','biofuel_elec_per_capita','biofuel_electricity','biofuel_share_elec','carbon_intensity_elec','coal_elec_per_capita','coal_electricity','coal_share_elec','electricity_demand','electricity_generation','per_capita_electricity','fossil_elec_per_capita','fossil_electricity','fossil_share_elec','gas_elec_per_capita','gas_electricity','gas_share_elec','greenhouse_gas_emissions','low_carbon_electricity','low_carbon_elec_per_capita','low_carbon_share_elec','nuclear_elec_per_capita','nuclear_electricity','nuclear_share_elec','oil_elec_per_capita','oil_electricity','oil_share_elec','renewables_elec_per_capita','renewables_electricity','renewables_share_elec']]

In [471]:
grp = data_cont.groupby('country')

In [473]:
grp.head()

,iso_code,country,year,population,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,carbon_intensity_elec,coal_elec_per_capita,coal_electricity,coal_share_elec,electricity_demand,electricity_generation,per_capita_electricity,fossil_elec_per_capita,fossil_electricity,fossil_share_elec,gas_elec_per_capita,gas_electricity,gas_share_elec,greenhouse_gas_emissions,low_carbon_electricity,low_carbon_elec_per_capita,low_carbon_share_elec,nuclear_elec_per_capita,nuclear_electricity,nuclear_share_elec,oil_elec_per_capita,oil_electricity,oil_share_elec,renewables_elec_per_capita,renewables_electricity,renewables_share_elec
247,None,Africa,2000,8.189524e+08,2.491,2.04,0.463,621.157,258.281,211.52,47.955,454.15,441.08,538.591,428.071,350.57,79.480,112.155,91.85,20.824,273.98,90.51,110.519,20.520,15.886,13.01,2.950,57.635,47.20,10.701,94.633,77.50,17.571
248,None,Africa,2001,8.394641e+08,2.502,2.10,0.457,619.706,258.165,216.72,47.168,462.61,459.46,547.325,435.313,365.43,79.535,120.291,100.98,21.978,284.73,94.03,112.012,20.465,12.770,10.72,2.333,56.858,47.73,10.388,99.242,83.31,18.132
249,None,Africa,2002,8.606118e+08,2.998,2.58,0.529,617.134,261.163,224.76,46.087,489.33,487.69,566.678,450.470,387.68,79.493,134.741,115.96,23.777,300.97,100.01,116.208,20.507,13.932,11.99,2.459,54.566,46.96,9.629,102.276,88.02,18.048
250,None,Africa,2003,8.823496e+08,2.867,2.53,0.497,625.682,269.462,237.76,46.698,510.48,509.14,577.028,465.360,410.61,80.648,140.534,124.00,24.355,318.56,98.53,111.668,19.352,14.348,12.66,2.487,55.364,48.85,9.595,97.320,85.87,16.866
251,None,Africa,2004,9.047816e+08,2.940,2.66,0.492,623.875,274.829,248.66,46.036,541.50,540.14,596.984,481.619,435.76,80.675,155.905,141.06,26.115,336.98,104.38,115.365,19.325,14.777,13.37,2.475,50.885,46.04,8.524,100.588,91.01,16.849
947,ATA,Antarctica,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
948,ATA,Antarctica,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
949,ATA,Antarctica,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
950,ATA,Antarctica,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
951,ATA,Antarctica,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### From the above results it is evident that negligible amount of data is available for the continent of 'Antarctica'. We can exclude it from our further analysis

In [476]:
data_cont = data_cont.loc[data["country"] != 'Antarctica']

In [478]:
maps = {'Africa':'AFR','Asia':'ASA','Europe':'EUR','North America':'NAE','South America':'SAM'}
data_cont['iso_code'] = data_cont['iso_code'].fillna(data_cont['country'].map(maps))

In [480]:
data_cont.isnull().sum(),data_cont.shape

(iso_code                      0
 country                       0
 year                          0
 population                    0
 biofuel_elec_per_capita       5
 biofuel_electricity           5
 biofuel_share_elec            5
 carbon_intensity_elec         5
 coal_elec_per_capita          5
 coal_electricity              5
 coal_share_elec               5
 electricity_demand            5
 electricity_generation        5
 per_capita_electricity        5
 fossil_elec_per_capita        5
 fossil_electricity            5
 fossil_share_elec             5
 gas_elec_per_capita           5
 gas_electricity               5
 gas_share_elec                5
 greenhouse_gas_emissions      5
 low_carbon_electricity        5
 low_carbon_elec_per_capita    5
 low_carbon_share_elec         5
 nuclear_elec_per_capita       5
 nuclear_electricity           5
 nuclear_share_elec            5
 oil_elec_per_capita           5
 oil_electricity               5
 oil_share_elec                5
 renewable

#### Out of the 120 rows only 5 in each energy statistic are null values. Hence, we can replace them with the mean value for that particular year.

In [483]:
iso_mapping_cont = data_cont[['country', 'iso_code']].drop_duplicates()

In [485]:
iso_mapping_cont

,country,iso_code
247,Africa,AFR
1304,Asia,ASA
6801,Europe,EUR
14130,North America,NAE
17823,South America,SAM


In [487]:
df_cont_grouped = data_cont.groupby(['country','year'], as_index=False).mean(numeric_only=True)

In [489]:
df_cont_final = df_cont_grouped.fillna(df_cont_grouped.mean(numeric_only=True))

In [491]:
df_cont_final_1 = df_cont_final.merge(iso_mapping_cont, on='country', how='left')

In [493]:
last_col = df_cont_final_1.columns[-1]

df_cont_final_1 = df_cont_final_1[[last_col] + list(df_cont_final_1.columns[:-1])]

In [495]:
df_cont_final_1.head()

,iso_code,country,year,population,biofuel_elec_per_capita,biofuel_electricity,biofuel_share_elec,carbon_intensity_elec,coal_elec_per_capita,coal_electricity,coal_share_elec,electricity_demand,electricity_generation,per_capita_electricity,fossil_elec_per_capita,fossil_electricity,fossil_share_elec,gas_elec_per_capita,gas_electricity,gas_share_elec,greenhouse_gas_emissions,low_carbon_electricity,low_carbon_elec_per_capita,low_carbon_share_elec,nuclear_elec_per_capita,nuclear_electricity,nuclear_share_elec,oil_elec_per_capita,oil_electricity,oil_share_elec,renewables_elec_per_capita,renewables_electricity,renewables_share_elec
0,AFR,Africa,2000,818952374.0,2.491,2.04,0.463,621.157,258.281,211.52,47.955,454.15,441.08,538.591,428.071,350.57,79.480,112.155,91.85,20.824,273.98,90.51,110.519,20.520,15.886,13.01,2.950,57.635,47.20,10.701,94.633,77.50,17.571
1,AFR,Africa,2001,839464127.0,2.502,2.10,0.457,619.706,258.165,216.72,47.168,462.61,459.46,547.325,435.313,365.43,79.535,120.291,100.98,21.978,284.73,94.03,112.012,20.465,12.770,10.72,2.333,56.858,47.73,10.388,99.242,83.31,18.132
2,AFR,Africa,2002,860611762.0,2.998,2.58,0.529,617.134,261.163,224.76,46.087,489.33,487.69,566.678,450.470,387.68,79.493,134.741,115.96,23.777,300.97,100.01,116.208,20.507,13.932,11.99,2.459,54.566,46.96,9.629,102.276,88.02,18.048
3,AFR,Africa,2003,882349569.0,2.867,2.53,0.497,625.682,269.462,237.76,46.698,510.48,509.14,577.028,465.360,410.61,80.648,140.534,124.00,24.355,318.56,98.53,111.668,19.352,14.348,12.66,2.487,55.364,48.85,9.595,97.320,85.87,16.866
4,AFR,Africa,2004,904781595.0,2.940,2.66,0.492,623.875,274.829,248.66,46.036,541.50,540.14,596.984,481.619,435.76,80.675,155.905,141.06,26.115,336.98,104.38,115.365,19.325,14.777,13.37,2.475,50.885,46.04,8.524,100.588,91.01,16.849


In [497]:
df_cont_final_1.isnull().sum()

iso_code                      0
country                       0
year                          0
population                    0
biofuel_elec_per_capita       0
biofuel_electricity           0
biofuel_share_elec            0
carbon_intensity_elec         0
coal_elec_per_capita          0
coal_electricity              0
coal_share_elec               0
electricity_demand            0
electricity_generation        0
per_capita_electricity        0
fossil_elec_per_capita        0
fossil_electricity            0
fossil_share_elec             0
gas_elec_per_capita           0
gas_electricity               0
gas_share_elec                0
greenhouse_gas_emissions      0
low_carbon_electricity        0
low_carbon_elec_per_capita    0
low_carbon_share_elec         0
nuclear_elec_per_capita       0
nuclear_electricity           0
nuclear_share_elec            0
oil_elec_per_capita           0
oil_electricity               0
oil_share_elec                0
renewables_elec_per_capita    0
renewabl

#### Now, that the required dataframes are ready, we will begin with creating the database in MySQL.
#### First, we will create a database and in it we create a table storing the unique iso_code and country combinations, 2nd table storing country and year wise data and the 3rd table storing the year-wise data for continents.

In [102]:
import mysql.connector

In [274]:
conn = mysql.connector.connect(host="localhost", user="root", password="Anirane15@isro", database="energy_database")
cursor = conn.cursor()
# Creating the table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS ISOCodes (
        iso_code VARCHAR(10) UNIQUE PRIMARY KEY,
        country_cont_name VARCHAR(255) UNIQUE
    )
""")

In [276]:
unique_countries = df_final[['country', 'iso_code']].drop_duplicates()
unique_countries = unique_countries.replace({np.nan: None})
for index, row in unique_countries.iterrows():
    cursor.execute("INSERT IGNORE INTO ISOCodes (country_cont_name, iso_code) VALUES (%s, %s)", (row['country'], row['iso_code']))

In [278]:
conn.commit()

### 1. Creating the data table for year-wise country data

In [281]:
# Creating a table for maintaining the various energy statistics
conn = mysql.connector.connect(host="localhost", user="root", password="Anirane15@isro", database="energy_database")
cursor = conn.cursor()
# Creating the 'country_energy_stats' table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS country_energy_stats (
        iso_code VARCHAR(10),
        year INT NOT NULL,
        population BIGINT NOT NULL,
        biofuel_elec_per_capita FLOAT, biofuel_electricity FLOAT, biofuel_share_elec FLOAT, carbon_intensity_elec FLOAT, coal_elec_per_capita FLOAT, coal_electricity FLOAT, coal_share_elec FLOAT, electricity_demand FLOAT,
        electricity_generation FLOAT, per_capita_electricity FLOAT, fossil_elec_per_capita FLOAT, fossil_electricity FLOAT, fossil_share_elec FLOAT, gas_elec_per_capita FLOAT, gas_electricity FLOAT, gas_share_elec FLOAT, greenhouse_gas_emissions FLOAT, low_carbon_electricity FLOAT, low_carbon_elec_per_capita FLOAT, low_carbon_share_elec FLOAT, nuclear_elec_per_capita FLOAT, nuclear_electricity FLOAT, nuclear_share_elec FLOAT, oil_elec_per_capita FLOAT, oil_electricity FLOAT, oil_share_elec FLOAT, renewables_elec_per_capita FLOAT, renewables_electricity FLOAT, renewables_share_elec FLOAT,
        FOREIGN KEY (iso_code) REFERENCES ISOCodes(iso_code),
        PRIMARY KEY (iso_code, year)
    )
""")

In [283]:
from tqdm import tqdm
def insert_energy_stats_bulk(df, cursor, conn, columns=None, batch_size=500):

    # Default energy columns if not specified
    if columns is None:
        columns = ['iso_code','year','population','biofuel_elec_per_capita','biofuel_electricity','biofuel_share_elec','carbon_intensity_elec','coal_elec_per_capita','coal_electricity','coal_share_elec','electricity_demand','electricity_generation','per_capita_electricity','fossil_elec_per_capita','fossil_electricity','fossil_share_elec','gas_elec_per_capita','gas_electricity','gas_share_elec','greenhouse_gas_emissions','low_carbon_electricity','low_carbon_elec_per_capita','low_carbon_share_elec','nuclear_elec_per_capita','nuclear_electricity','nuclear_share_elec','oil_elec_per_capita','oil_electricity','oil_share_elec','renewables_elec_per_capita','renewables_electricity','renewables_share_elec']
    
    # Keep only relevant columns
    energy_df = df[columns].copy()
    
    # Drop rows without iso_code or year (essential keys)
    energy_df = energy_df.dropna(subset=['iso_code', 'year'])
    energy_df = energy_df.replace({np.nan: None})
    
    print("Inserting energy stats into database...")

    placeholders = ', '.join(['%s'] * len(columns))
    col_names = ', '.join(columns)
    
    count = 0

    for _, row in tqdm(energy_df.iterrows(), total=len(energy_df)):
        try:
            values = [row[col] for col in columns]
            cursor.execute(f"""
                INSERT INTO country_energy_stats ({col_names})
                VALUES ({placeholders})
            """, values)
            count += 1

            if count % batch_size == 0:
                conn.commit()

        except Exception as e:
            print(f"Error inserting row: {e}")

    conn.commit()
    print(f"Inserted successfully. Total rows inserted: {count}")

In [285]:
sel_cols = ['iso_code','year','population','biofuel_elec_per_capita','biofuel_electricity','biofuel_share_elec','carbon_intensity_elec','coal_elec_per_capita','coal_electricity','coal_share_elec','electricity_demand','electricity_generation','per_capita_electricity','fossil_elec_per_capita','fossil_electricity','fossil_share_elec','gas_elec_per_capita','gas_electricity','gas_share_elec','greenhouse_gas_emissions','low_carbon_electricity','low_carbon_elec_per_capita','low_carbon_share_elec','nuclear_elec_per_capita','nuclear_electricity','nuclear_share_elec','oil_elec_per_capita','oil_electricity','oil_share_elec','renewables_elec_per_capita','renewables_electricity','renewables_share_elec']
insert_energy_stats_bulk(df_final, cursor, conn, columns=sel_cols)

Inserting energy stats into database...


100%|████████████████████████████████████████████████████████████████████████████| 5118/5118 [00:02<00:00, 2179.15it/s]

Inserted successfully. Total rows inserted: 5118


### 2. Creating the data table for year-wise continents data

In [288]:
# Inserting the ISO mapping created for continents earlier into SQL table ISOCodes (Note: It already exists for Australia so doing only for the other 5.)
conn = mysql.connector.connect(host="localhost", user="root", password="Anirane15@isro", database="energy_database")
cursor = conn.cursor()
# Inserting the values in the isocodes table
cursor.executemany("INSERT IGNORE INTO ISOCodes (country_cont_name, iso_code) VALUES (%s, %s)", [
    ('Africa', 'AFR'),
    ('Asia', 'ASA'),
    ('North America', 'NAE'),
    ('South America', 'SAM'),
    ('Europe', 'EUR')])

In [290]:
conn.commit()

In [292]:
# Creating a table for maintaining the various 'CONTINENTAL' energy statistics
conn = mysql.connector.connect(host="localhost", user="root", password="Anirane15@isro", database="energy_database")
cursor = conn.cursor()
# Creating the 'continent_energy' table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS continent_energy (
        iso_code VARCHAR(10),
        year INT NOT NULL,
        population BIGINT NOT NULL,
        biofuel_elec_per_capita FLOAT, biofuel_electricity FLOAT, biofuel_share_elec FLOAT, carbon_intensity_elec FLOAT, coal_elec_per_capita FLOAT, coal_electricity FLOAT, coal_share_elec FLOAT, electricity_demand FLOAT,
        electricity_generation FLOAT, per_capita_electricity FLOAT, fossil_elec_per_capita FLOAT, fossil_electricity FLOAT, fossil_share_elec FLOAT, gas_elec_per_capita FLOAT, gas_electricity FLOAT, gas_share_elec FLOAT, greenhouse_gas_emissions FLOAT, low_carbon_electricity FLOAT, low_carbon_elec_per_capita FLOAT, low_carbon_share_elec FLOAT, nuclear_elec_per_capita FLOAT, nuclear_electricity FLOAT, nuclear_share_elec FLOAT, oil_elec_per_capita FLOAT, oil_electricity FLOAT, oil_share_elec FLOAT, renewables_elec_per_capita FLOAT, renewables_electricity FLOAT, renewables_share_elec FLOAT,
        FOREIGN KEY (iso_code) REFERENCES ISOCodes(iso_code),
        PRIMARY KEY (iso_code, year)
    )
""")

In [294]:
def insert_cont_energy_stats_bulk(df, cursor, conn, columns=None, batch_size=500):

    # Default energy columns if not specified
    if columns is None:
        columns = ['iso_code','year','population','biofuel_elec_per_capita','biofuel_electricity','biofuel_share_elec','carbon_intensity_elec','coal_elec_per_capita','coal_electricity','coal_share_elec','electricity_demand','electricity_generation','per_capita_electricity','fossil_elec_per_capita','fossil_electricity','fossil_share_elec','gas_elec_per_capita','gas_electricity','gas_share_elec','greenhouse_gas_emissions','low_carbon_electricity','low_carbon_elec_per_capita','low_carbon_share_elec','nuclear_elec_per_capita','nuclear_electricity','nuclear_share_elec','oil_elec_per_capita','oil_electricity','oil_share_elec','renewables_elec_per_capita','renewables_electricity','renewables_share_elec']
    
    # Keep only relevant columns
    energy_df = df[columns].copy()
    
    # Drop rows without iso_code or year (essential keys)
    energy_df = energy_df.dropna(subset=['iso_code', 'year'])
    energy_df = energy_df.replace({np.nan: None})
    
    print("Inserting energy stats into database...")

    placeholders = ', '.join(['%s'] * len(columns))
    col_names = ', '.join(columns)
    
    count = 0

    for _, row in tqdm(energy_df.iterrows(), total=len(energy_df)):
        try:
            values = [row[col] for col in columns]
            cursor.execute(f"""
                INSERT INTO continent_energy ({col_names})
                VALUES ({placeholders})
            """, values)
            count += 1

            if count % batch_size == 0:
                conn.commit()

        except Exception as e:
            print(f"Error inserting row: {e}")

    conn.commit()
    print(f"Inserted successfully. Total rows inserted: {count}")

In [296]:
sel_cols = ['iso_code','year','population','biofuel_elec_per_capita','biofuel_electricity','biofuel_share_elec','carbon_intensity_elec','coal_elec_per_capita','coal_electricity','coal_share_elec','electricity_demand','electricity_generation','per_capita_electricity','fossil_elec_per_capita','fossil_electricity','fossil_share_elec','gas_elec_per_capita','gas_electricity','gas_share_elec','greenhouse_gas_emissions','low_carbon_electricity','low_carbon_elec_per_capita','low_carbon_share_elec','nuclear_elec_per_capita','nuclear_electricity','nuclear_share_elec','oil_elec_per_capita','oil_electricity','oil_share_elec','renewables_elec_per_capita','renewables_electricity','renewables_share_elec']
insert_cont_energy_stats_bulk(df_cont_final_1, cursor, conn, columns=sel_cols)

Inserting energy stats into database...


100%|██████████████████████████████████████████████████████████████████████████████| 120/120 [00:00<00:00, 1087.01it/s]

Inserted successfully. Total rows inserted: 120


## With this we have finished creating the MySQL database and inserting all the data.